In [1]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [2]:
from web3 import AsyncWeb3, Web3
from web3.eth import Contract
from datetime import datetime


import json


def read_abi(token: str):
    with open(f"../config/{token.lower()}_abi.json") as f:
        data = json.load(f)
        return data

In [3]:
w3 = Web3(Web3.HTTPProvider("https://bitter-wandering-feather.arbitrum-mainnet.quiknode.pro/862a558ad28be94cf6b1ccae509bdca74a19086a"))

rockonyx_delta_neutral_vault_abi = read_abi("rockonyxdeltaneutralvault")
vault_contract = w3.eth.contract(
    address="0xF12F8753C25F8005311a24bbbA1f7209249B00e3",
    abi=rockonyx_delta_neutral_vault_abi,
)


In [4]:

def get_shares(vault_contract: Contract, user: str, decimals = 1e6, block_number=200199390):
    tvl = vault_contract.functions.balanceOf(user).call(block_identifier=block_number)

    return tvl / decimals

def get_pps(vault_contract: Contract, decimals = 1e6, block_number=200199390):
    tvl = vault_contract.functions.pricePerShare().call(block_identifier=block_number)

    return tvl / decimals
# get_shares(vault_contract, Web3.to_checksum_address("0x9C70305f78a715cb79299a93C5E826310b86865E"), block_number=287646145)

In [5]:

import pandas as pd

# Read the CSV file
df = pd.read_csv('data/deposits-hype-vault.csv')

# Display the first few rows to verify the data
df.head()


,Transaction Hash,Status,Method,Block,Date Time (UTC),From,Amount
0,0xf6883ae21052c0d9ef373f098cc806fcdcd44cb636b8...,Success,Deposit,289077970,12/27/2024 7:47,0x7E34D886E2D2283ddC1274dDb5b0583FC423A6AE,20
1,0xd0b2fe3412e7c15cc3e4a1babdc940c41d7039867f9f...,Success,Deposit,289077134,12/27/2024 7:44,0x0d6F65f2861f545A31B3DBeaba681C3dD235663E,362.74
2,0xa10640403d28211ec92b08796c7892a54b8eab190838...,Success,Deposit,289073727,12/27/2024 7:29,0x25a5e5A62F788834d87864833aa06Ee7b7A966EA,199.956985
3,0x495e5452d8c56b18c455b6bd1476cb6198129dee2e9d...,Success,Deposit,289039773,12/27/2024 5:08,0xADd9e7Df46f6177a1Fc5408914355B418F56D38A,400
4,0x5683476d8fd661b5f50bb4f017cbe08415780d8c8ec8...,Success,Deposit,289024483,12/27/2024 4:04,0x18fCf0934e053AC5f4B0a75621Dbd0BcAC8c38Fd,"15,000"


In [6]:
# import uuid

# # Generating INSERT statements
# vault_slug = 'hype-delta-neutral-v1'
# created_at = datetime.utcnow().isoformat()

# insert_statements = []
# for wallet_address in df['From']:
#     insert_statements.append(f"""INSERT INTO config.whitelist_wallets(
#     id, vault_slug, wallet_address, created_at)
#     VALUES ('{uuid.uuid4()}', '{vault_slug}', '{wallet_address}', '{created_at}');""")

# # Display the results
# for statement in insert_statements:
#     print(statement)

INSERT INTO config.whitelist_wallets(
    id, vault_slug, wallet_address, created_at)
    VALUES ('17f330ea-75d2-4fe6-a0d0-74c66788a868', 'hype-delta-neutral-v1', '0x7E34D886E2D2283ddC1274dDb5b0583FC423A6AE', '2024-12-29T08:11:49.080083');
INSERT INTO config.whitelist_wallets(
    id, vault_slug, wallet_address, created_at)
    VALUES ('ea2ab1a8-245e-4b18-887f-0f1fe59cfd48', 'hype-delta-neutral-v1', '0x0d6F65f2861f545A31B3DBeaba681C3dD235663E', '2024-12-29T08:11:49.080083');
INSERT INTO config.whitelist_wallets(
    id, vault_slug, wallet_address, created_at)
    VALUES ('db6e02fd-1123-4745-bd18-d75fb24b6f3e', 'hype-delta-neutral-v1', '0x25a5e5A62F788834d87864833aa06Ee7b7A966EA', '2024-12-29T08:11:49.080083');
INSERT INTO config.whitelist_wallets(
    id, vault_slug, wallet_address, created_at)
    VALUES ('b8e0d056-0da9-4776-a478-8056c7d2c789', 'hype-delta-neutral-v1', '0xADd9e7Df46f6177a1Fc5408914355B418F56D38A', '2024-12-29T08:11:49.080083');
INSERT INTO config.whitelist_wallets(
  

In [8]:
# Create a new column with shares calculation
df['Shares'] = df.apply(
    lambda row: get_shares(
        vault_contract,
        Web3.to_checksum_address(row['From']),
        block_number=int(row['Block'])
    ),
    axis=1
)

# Create a new column with shares calculation
df['PPS'] = df.apply(
    lambda row: get_pps(
        vault_contract,
        block_number=int(row['Block'])
    ),
    axis=1
)

# Display the results
df[['From', 'Block', 'Shares', 'PPS']].head()

,From,Block,Shares,PPS
0,0x7E34D886E2D2283ddC1274dDb5b0583FC423A6AE,289077970,19.932965,1.003363
1,0x0d6F65f2861f545A31B3DBeaba681C3dD235663E,289077134,361.524194,1.003363
2,0x25a5e5A62F788834d87864833aa06Ee7b7A966EA,289073727,199.286783,1.003363
3,0xADd9e7Df46f6177a1Fc5408914355B418F56D38A,289039773,398.659308,1.003363
4,0x18fCf0934e053AC5f4B0a75621Dbd0BcAC8c38Fd,289024483,14949.724077,1.003363


In [11]:
# Order df by 'Date Time (UTC)' in ascending order
df = df.sort_values(by='Date Time (UTC)', ascending=True)

df



,Transaction Hash,Status,Method,Block,Date Time (UTC),From,Amount,Shares,PPS
83,0x7d2bc52b999ff597092eda8e4fd448c418a01a6102d0...,Success,Deposit,288037720,12/24/2024 7:18,0xBC05da14287317FE12B1a2b5a0E1d756Ff1801Aa,80,80.000000,1.000000
82,0xcef85f9fb49f09462e73bc9a0043bdd0ad206d2a6a5b...,Success,Deposit,288512122,12/25/2024 16:22,0xc8cdE2212bf6fDfDE6bcC7df0A3f0f4B82fDd240,10,9.964992,1.003513
81,0x0bbc6c8bc31dc322d4f6b44a8829fa542497896d7eb6...,Success,Deposit,288516760,12/25/2024 16:42,0xFf42F643b7E439cbFEaC5DC98843150784047DF7,10,9.964992,1.003513
80,0x073f6d71cae465cd9a8d6164673b167e55e5e20d1ab1...,Success,Deposit,288517190,12/25/2024 16:44,0xFf42F643b7E439cbFEaC5DC98843150784047DF7,582.83197,590.756640,1.003513
79,0xd97c7234a626144fa3c8d9f4c5325ff699f236c62a57...,Success,Deposit,288530962,12/25/2024 17:41,0x675bC14456dAeD7662F4d77E61eF0478393FC350,"1,005",1001.481794,1.003513
...,...,...,...,...,...,...,...,...,...
4,0x5683476d8fd661b5f50bb4f017cbe08415780d8c8ec8...,Success,Deposit,289024483,12/27/2024 4:04,0x18fCf0934e053AC5f4B0a75621Dbd0BcAC8c38Fd,"15,000",14949.724077,1.003363
3,0x495e5452d8c56b18c455b6bd1476cb6198129dee2e9d...,Success,Deposit,289039773,12/27/2024 5:08,0xADd9e7Df46f6177a1Fc5408914355B418F56D38A,400,398.659308,1.003363
2,0xa10640403d28211ec92b08796c7892a54b8eab190838...,Success,Deposit,289073727,12/27/2024 7:29,0x25a5e5A62F788834d87864833aa06Ee7b7A966EA,199.956985,199.286783,1.003363
1,0xd0b2fe3412e7c15cc3e4a1babdc940c41d7039867f9f...,Success,Deposit,289077134,12/27/2024 7:44,0x0d6F65f2861f545A31B3DBeaba681C3dD235663E,362.74,361.524194,1.003363


In [16]:
# ... existing code ...
# Remove commas and convert 'Amount' to float
df['Amount'] = df['Amount'].str.replace(',', '').astype(float)

# Tính tổng 'Amount', giữ 'Shares' mới nhất và tính median cho 'PPS'
df_grouped = df.groupby('From').agg(
    Total_Amount=('Amount', 'sum'),
    Latest_Shares=('Shares', 'last'),  # Lấy giá trị cuối cùng của Shares
    Median_PPS=('PPS', 'median')
).reset_index()

# Hiển thị kết quả
df_grouped.head()
# ... existing code ...

,From,Total_Amount,Latest_Shares,Median_PPS
0,0x015a0138Ecc957859853908Ad455647BD33e635B,635.631880,633.532353,1.003314
1,0x055BA87Dbff972e23bCF26ea4728C31e05240e66,99.000000,98.603318,1.004023
2,0x058F21756ee0Ed7DAB5158661E92AbEEeef6c9a7,10.000000,9.970189,1.002990
3,0x0806A4bb6034157477b571369B87105391Dd1687,7844.553047,7812.311946,1.004127
4,0x095c938780605911Df64dc66342C3260989E68A9,15765.007882,15701.839381,1.004023


In [17]:
df_grouped.to_csv('./data/deposit-hype-vault-2.csv', index=False)